In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
z=torch.FloatTensor([1,2,3])

In [4]:
h = F.softmax(z,dim =0)
print(h)

tensor([0.0900, 0.2447, 0.6652])


In [6]:
print(h.sum())

tensor(1.)


In [7]:
z= torch.rand(3,5,requires_grad =True)
h = F.softmax(z,dim=1)
print(h)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [11]:
y = torch.randint(5,(3,)).long()
print(y)

tensor([2, 1, 0])


In [13]:
y_one_hot = torch.zeros_like(h)
print(y_one_hot)
print(y.unsqueeze(1))
y_one_hot.scatter_(1,y.unsqueeze(1),1)#

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[2],
        [1],
        [0]])


tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [19]:
cost = (y_one_hot*-torch.log(h)).sum(dim=1).mean()
print(cost)

tensor(1.6682, grad_fn=<MeanBackward0>)


# torch.nn.Functional 을 사용한 Cross-entropy Loss

In [23]:
F.nll_loss(F.log_softmax(z,dim=1),y)

tensor(1.6682, grad_fn=<NllLossBackward0>)

In [27]:
F.cross_entropy(z,y)

tensor(1.6682, grad_fn=<NllLossBackward0>)

# Low Level Cross-entropy

In [25]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [29]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    h= F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(h)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(h, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


# F.cross_entropy를 사용한 방법

In [30]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    z = x_train.matmul(W) + b 
    cost = F.cross_entropy(z, y_train)


    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


# nn.Module을 사용한 Cross-entropy

In [33]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        print("TT")
        return self.linear(x)

In [34]:
model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)

    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

TT
Epoch    0/1000 Cost: 1.777960
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
Epoch  100/1000 Cost: 0.654127
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
Epoch  200/1000 Cost: 0.561501
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
TT
Epoc